In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib
from io import StringIO
from scipy.spatial.distance import pdist, squareform
from tqdm import tqdm

matplotlib.rcParams['pdf.fonttype'] = 42

strain = 0.30

In [2]:
def data_org(file_path):

    buffer = StringIO()
    current_time = None
    
    with open(file_path, "r") as f:
        for line in f:
            if line.strip() == "ITEM: TIMESTEP":
                current_time = f.readline().strip()
                continue
                
            parts = line.strip().split()
            if len(parts) == 5:
                buffer.write(f"{current_time} {' '.join(parts)}\n")
    
    buffer.seek(0)
    df = pd.read_csv(buffer, sep=' ', 
                     names=["time", "id", "type", "x", "y", "z"])
    return df

In [3]:

def contact_index(lamins_df, chromatin_df, time):

    last_chromatin = chromatin_df.loc[chromatin_df["time"] == time].reset_index(drop=True)
    last_lamins = lamins_df.loc[lamins_df["time"] == time].reset_index(drop=True)
    
    chromatin_coords = last_chromatin[["x", "y", "z"]].to_numpy()
    lamins_coords = last_lamins[["x", "y", "z"]].to_numpy()
    
    contact_distances = []

    for chromatin in chromatin_coords:

        distances = np.sqrt(np.sum((chromatin - lamins_coords)**2, axis=1))
        contact_distances.append(np.min(distances))
    
    last_chromatin["contact index"] = contact_distances
    
    return last_chromatin

In [4]:
import warnings

warnings.filterwarnings('ignore')

def calculate_contacts(lamins_df,chromatin_df, time):

    last_chromatin_df = chromatin_df[chromatin_df["time"] == time]

    last_chromatin_df = last_chromatin_df.sort_values(by = "id")

    last_chromatin_df.reset_index(drop = True, inplace = True)


    last_lamins_df = lamins_df[lamins_df["time"] == time]

    last_lamins_df = last_lamins_df.sort_values(by = "id")

    last_lamins_df.reset_index(drop = True, inplace = True)

    lamin_map = np.array([])
    lamin_non_map = np.array([])
    
    for ind, lam in last_lamins_df.iterrows():
        distance = (last_chromatin_df["x"] - lam["x"])**2 + (last_chromatin_df["y"] - lam["y"])**2 + (last_chromatin_df["z"] - lam["z"])**2
        distance = distance**0.5
        lamin_map = np.append(lamin_map, len(distance[distance <= 2.5]))
        lamin_non_map = np.append(lamin_non_map, len(distance[distance > 2.5]))
        
    last_lamins_df["chromatin contacts"] = lamin_map

    last_lamins_df["non chromatin contacts"] = lamin_non_map

    return last_lamins_df

In [5]:
def plot_contacts(last_lamins_df):

    scaler = MinMaxScaler()

    last_lamins_df['norm. chromatin contacts'] = scaler.fit_transform(last_lamins_df['chromatin contacts'].values.reshape(-1, 1))
    last_lamins_df['norm. non chromatin contacts'] = scaler.fit_transform(last_lamins_df['non chromatin contacts'].values.reshape(-1, 1))

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    sns.kdeplot(data=last_lamins_df, x='norm. chromatin contacts', fill=True, alpha=0.5, label='chromatin contacts', ax=ax)
    sns.kdeplot(data=last_lamins_df, x='norm. non chromatin contacts', fill=True, alpha=0.5, label='non chromatin contacts', ax=ax)
    ax.set_xlabel('Normalized')
    ax.set_ylabel('Density')
    ax.set_title('Contacts')
    ax.legend(loc='upper right')

    plt.show()    

In [23]:

def ps_func(chr_ind_last):

    k_step = 4
    k_s = np.unique(np.round([k_step * (1.11785**i) for i in range(65)]).astype(int))
    max_k = k_s.max()

    chr_length = 6002
    num_chr = 8
    
    genomic_dist = np.zeros(len(k_s), dtype=float)
    coords = chr_ind_last[['x', 'y', 'z']].values.reshape(-1, 3)
    
    for chr_n in range(num_chr):
        
        start = chr_n * chr_length
        end = start + chr_length
        chr_coords = coords[start+1000:end]
        valid_length = len(chr_coords)
        
        for i in range(valid_length):

            if i  >= chr_length-4:
                continue
                
            diffs = chr_coords[i] - chr_coords[i:]

            distances = np.sqrt(np.sum(diffs**2, axis=1))
            
            for s, k in enumerate(k_s):

                if k > len(distances):
                    continue
                
                contacts = np.sum(distances[:int(k)] <= 2.5)
                
                genomic_dist[s] += contacts / int(k)

    genomic_dist /= ((chr_length-1000)*num_chr)
    
    return np.column_stack((k_s, genomic_dist))
    


In [24]:
def process_df(df):

    is_lamin = df["type"].isin([1, 9, 10])
    coords = df[["x", "y", "z"]].values
    
    radial = np.linalg.norm(coords, axis=1)
    
    lamin_df = df[is_lamin].copy()
    chromatin_df = df[~is_lamin].copy()
    
    lamin_df["lamin radial_distance"] = radial[is_lamin]
    chromatin_df["chromatin radial_distance"] = radial[~is_lamin]
    
    return (
        lamin_df.sort_values("id", ignore_index=True),
        chromatin_df.sort_values("id", ignore_index=True)
    )

In [25]:
def strain_fc(lamins_df):

    max_y_initial = lamins_df.loc[lamins_df['time'] == 0, 'y'].max()
    
    max_y_per_time = lamins_df.groupby('time', sort=True)['y'].max().reset_index()
    
    max_y_per_time['strain'] = (max_y_per_time['y'] - max_y_initial) / max_y_initial
    
    return max_y_per_time[['time', 'strain']]

In [26]:
from tqdm import tqdm

def process_pulling_data(base_path, condition, output_path, strain):

    ps_data_last_array = np.zeros((63, 111))

    ps_data_first_array = np.zeros((63, 11))

    before_contacts = np.zeros((48016, 10))

    after_contacts = np.zeros((48016, 110))

    config = 1

    for rep in tqdm(range(1, 11)):

        path = f'{base_path}/r{rep}/dump.pulling'

        df = data_org(path)

        lamins_df, chromatin_df = process_df(df)

        time_strain = strain_fc(lamins_df)

        time_strain.sort_values(by = 'time', inplace = True)

        closest_time = time_strain[(time_strain['strain'] > strain - 0.01) & (time_strain['strain'] < strain + 0.01)].iloc[0]['time']

        closes_time_list = [closest_time - 5000, closest_time - 4000, closest_time - 3000, closest_time - 2000, closest_time - 1000, closest_time, closest_time + 1000, 
                            closest_time + 2000, closest_time + 3000, closest_time + 4000, closest_time + 5000]

        chr_ind_0 = contact_index(lamins_df, chromatin_df, 0)   

        before_contacts[:,rep-1] = chr_ind_0['contact index']

        ps_data_first = ps_func(chr_ind_0)

        for ind, time in (enumerate(closes_time_list)):

            chr_ind_last = contact_index(lamins_df, chromatin_df, time)   

            ps_data_last = ps_func(chr_ind_last)
        
            ps_data_last_array[:,config] = ps_data_last[:,1]

            after_contacts[:,config-1] = chr_ind_last['contact index']

            config += 1

            print(config)

        ps_data_first_array[:,rep] = ps_data_first[:,1]
    
    ps_data_last_array[:,0] = ps_data_last[:,0]

    ps_data_first_array[:,0] = ps_data_first[:,0]

    pd.DataFrame(ps_data_first_array).to_csv(f'{output_path}/ps_first_{condition}.csv', index=False)
    pd.DataFrame(ps_data_last_array).to_csv(f'{output_path}/ps_last_{condition}.csv', index=False)

    pd.DataFrame(before_contacts).to_csv(f'{output_path}/before_contacts_{condition}.csv', index=False)
    pd.DataFrame(after_contacts).to_csv(f'{output_path}/after_contacts_{condition}.csv', index=False)
   
    return ps_data_first_array, ps_data_last_array

In [27]:
def process_and_plot_ps_data(ps_data_first, ps_data_last, output_path,  name, b_color, color):

    columns = ['Genomic Distance'] + [f'P(s)_{i}' for i in range(1, 111)]

    columns_first = ['Genomic Distance'] + [f'P(s)_{i}' for i in range(1, 11)]
    
    PSFS_FTNC_s = pd.DataFrame(ps_data_first, columns=columns_first)
    PSLS_FTNC_s = pd.DataFrame(ps_data_last, columns=columns)

    for df in [PSFS_FTNC_s, PSLS_FTNC_s]:
        df['mean'] = df.iloc[:, 1:].mean(axis=1)
        n = df.iloc[:, 1:111].shape[1]
        df['sd'] = df.iloc[:, 1:].std(axis=1) / np.sqrt(n)

    PSFS_FTNC_melt_s = PSFS_FTNC_s.melt(id_vars=['Genomic Distance', 'mean', 'sd'], var_name='rep', value_name='P(s)')
    PSLS_FTNC_melt_s = PSLS_FTNC_s.melt(id_vars=['Genomic Distance', 'mean', 'sd'], var_name='rep', value_name='P(s)')

    fig, axes = plt.subplots(1, 1, figsize=(3, 3), dpi=300)

    sns.lineplot(data=PSFS_FTNC_melt_s, x='Genomic Distance', y='P(s)', errorbar="se", color=b_color, linewidth=0.5)
    sns.lineplot(data=PSLS_FTNC_melt_s, x='Genomic Distance', y='P(s)', errorbar="se", linewidth=0.5, color=color)    

    axes.set_xscale('log')
    axes.set_yscale('log')
    axes.set_xlim((0, 6002))
    axes.set_title(f'P(s) vs Genomic Distance')

    plt.tight_layout()
    plt.savefig(f'{output_path}/PS_{name}.pdf', bbox_inches='tight')
    plt.close()

    return PSFS_FTNC_s, PSLS_FTNC_s, PSFS_FTNC_melt_s, PSLS_FTNC_melt_s


In [28]:
def filter_and_plot_zoomed_ps_data(PSFS_FTNC_melt_s, PSLS_FTNC_melt_s, base_path,  condition, output_path, b_color, color):

    PSFS_FTNC_melt_s_filtered = PSFS_FTNC_melt_s[
        (PSFS_FTNC_melt_s['Genomic Distance'] >= 100) & 
        (PSFS_FTNC_melt_s['Genomic Distance'] <= 2000) &
        (PSFS_FTNC_melt_s['P(s)'] <= 0.1)
    ]

    PSLS_FTNC_melt_s_filtered = PSLS_FTNC_melt_s[
        (PSLS_FTNC_melt_s['Genomic Distance'] >= 100) & 
        (PSLS_FTNC_melt_s['Genomic Distance'] <= 2000) &
        
        (PSLS_FTNC_melt_s['P(s)'] <= 0.1)
    ]

    fig, axes = plt.subplots(1, 1, figsize=(4, 4), dpi=300)

    sns.lineplot(data=PSFS_FTNC_melt_s_filtered, x='Genomic Distance', y='P(s)', 
                 errorbar="se", color=b_color, linewidth=0.5)
    sns.lineplot(data=PSLS_FTNC_melt_s_filtered, x='Genomic Distance', y='P(s)', 
                 errorbar="se", linewidth=0.5, color=color)    

    axes.set_xscale('log')
    axes.set_yscale('log')
    axes.set_xlim((100, 1000))
    axes.set_ylim((0.01, 0.1))
    axes.set_xlabel('')
    axes.set_ylabel('')
    axes.set_title(f'Zoomed P(s) vs Genomic Distance')

    plt.tight_layout()
    plt.savefig(f'{output_path}/PS_{condition}_zoom.pdf', bbox_inches='tight')
    plt.close()


In [29]:
base_path = '/Users/attar/Desktop/PS/New Folder With Items/FacTether-NoCross'

output_path = '/Users/attar/Desktop/PS/New Folder With Items/Results'

condition = base_path.split('/')[-1]

color = '#FF0000'

b_color = '#0B3D91'

ps_data_first_array, ps_data_last_array = process_pulling_data(base_path, condition, output_path, strain)

PSFS_FTNC_s_het, PSLS_FTNC_s_het,  PSFS_FTNC_melt_s_het, PSLS_FTNC_melt_s_het = process_and_plot_ps_data(ps_data_first_array, ps_data_last_array, output_path, condition, b_color, color)

filter_and_plot_zoomed_ps_data(PSFS_FTNC_melt_s_het, PSLS_FTNC_melt_s_het, base_path, condition, output_path,  b_color, color)

  0%|          | 0/10 [00:00<?, ?it/s]

2
3
4
5
6
7
8
9
10
11


 10%|█         | 1/10 [01:11<10:43, 71.53s/it]

12
13
14
15
16
17
18
19
20
21
22


 20%|██        | 2/10 [02:22<09:31, 71.48s/it]

...

89
90
91
92
93
94
95
96
97
98
99


 90%|█████████ | 9/10 [10:43<01:11, 71.15s/it]

100
101
102
103
104
105
106
107
108
109
110


100%|██████████| 10/10 [11:54<00:00, 71.41s/it]

111


In [30]:
base_path = '/Users/attar/Desktop/PS/New Folder With Items/NoTether-NoCross'

output_path = '/Users/attar/Desktop/PS/New Folder With Items/Results'

condition = base_path.split('/')[-1]

color = '#A0A0A4'

ps_data_first_array, ps_data_last_array = process_pulling_data(base_path, condition, output_path, strain)

PSFS_FTNC_s_het, PSLS_FTNC_s_het,  PSFS_FTNC_melt_s_het, PSLS_FTNC_melt_s_het = process_and_plot_ps_data(ps_data_first_array, ps_data_last_array, output_path, condition,  b_color, color)

filter_and_plot_zoomed_ps_data(PSFS_FTNC_melt_s_het, PSLS_FTNC_melt_s_het, base_path, condition, output_path,  b_color, color)


  0%|          | 0/10 [00:00<?, ?it/s]

2
3
4
5
6
7
8
9
10
11


 10%|█         | 1/10 [01:10<10:31, 70.18s/it]

12
13
14
15
16
17
18
19
20
21
22


 20%|██        | 2/10 [02:20<09:22, 70.34s/it]

...

89
90
91
92
93
94
95
96
97
98
99


 90%|█████████ | 9/10 [10:51<01:13, 73.84s/it]

100
101
102
103
104
105
106
107
108
109
110


100%|██████████| 10/10 [12:07<00:00, 72.73s/it]

111


In [33]:
PSLS_FTNC_s_het

,Genomic Distance,P(s)_1,P(s)_2,P(s)_3,P(s)_4,P(s)_5,P(s)_6,P(s)_7,P(s)_8,P(s)_9,...,P(s)_103,P(s)_104,P(s)_105,P(s)_106,P(s)_107,P(s)_108,P(s)_109,P(s)_110,mean,sd
0,4.0,0.936906,0.937612,0.937519,0.936500,0.937825,0.937369,0.936857,0.937506,0.937531,...,0.937050,0.936176,0.936925,0.936850,0.936994,0.936663,0.935813,0.936550,0.936875,5.479198e-05
1,5.0,0.839819,0.840884,0.841089,0.839429,0.841179,0.841758,0.839454,0.840514,0.841558,...,0.841368,0.839454,0.841288,0.840584,0.840274,0.839929,0.839504,0.839564,0.840500,9.009738e-05
2,6.0,0.751579,0.752649,0.753265,0.751254,0.752457,0.753574,0.750841,0.752507,0.753869,...,0.753669,0.751341,0.753811,0.752799,0.752516,0.751645,0.751883,0.751166,0.752619,1.223897e-04
3,7.0,0.677793,0.678546,0.679253,0.676972,0.678354,0.679303,0.676951,0.678450,0.679689,...,0.679999,0.677193,0.680278,0.678404,0.678721,0.677483,0.678015,0.676686,0.678586,1.467678e-04
4,8.0,0.616266,0.617100,0.617903,0.615510,0.616372,0.617659,0.615204,0.617362,0.617965,...,0.618225,0.615354,0.618762,0.616569,0.617044,0.615794,0.616860,0.615060,0.616984,1.593899e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,3199.0,0.001406,0.001396,0.001394,0.001401,0.001413,0.001428,0.001405,0.001424,0.001417,...,0.001383,0.001372,0.001392,0.001373,0.001368,0.001381,0.001385,0.001392,0.001413,3.779931e-06
59,3576.0,0.001008,0.001010,0.001009,0.001008,0.001023,0.001033,0.001017,0.001028,0.001027,...,0.000971,0.000969,0.000982,0.000969,0.000962,0.000970,0.000977,0.000980,0.001023,3.395254e-06
60,3998.0,0.000664,0.000662,0.000664,0.000659,0.000673,0.000676,0.000664,0.000672,0.000665,...,0.000616,0.000618,0.000626,0.000619,0.000616,0.000622,0.000624,0.000630,0.000670,2.488933e-06
61,4469.0,0.000315,0.000313,0.000318,0.000312,0.000320,0.000325,0.000319,0.000323,0.000317,...,0.000313,0.000316,0.000316,0.000315,0.000310,0.000314,0.000314,0.000319,0.000325,1.839189e-06


In [31]:
base_path = '/Users/attar/Desktop/PS/New Folder With Items/DoubleCross'

output_path = '/Users/attar/Desktop/PS/New Folder With Items/Results'

condition = base_path.split('/')[-1]

b_color = '#0B3D91'

color = '#077E97'

ps_data_first_array, ps_data_last_array = process_pulling_data(base_path, condition, output_path, strain)

PSFS_FTNC_s_het, PSLS_FTNC_s_het,  PSFS_FTNC_melt_s_het, PSLS_FTNC_melt_s_het = process_and_plot_ps_data(ps_data_first_array, ps_data_last_array, output_path, condition,  b_color, color)

filter_and_plot_zoomed_ps_data(PSFS_FTNC_melt_s_het, PSLS_FTNC_melt_s_het, base_path, condition, output_path,  b_color, color)


  0%|          | 0/10 [00:00<?, ?it/s]

2
3
4
5
6
7
8
9
10
11


 10%|█         | 1/10 [01:10<10:32, 70.23s/it]

12
13
14
15
16
17
18
19
20
21
22


 20%|██        | 2/10 [02:22<09:32, 71.62s/it]

...

89
90
91
92
93
94
95
96
97
98
99


 90%|█████████ | 9/10 [10:49<01:11, 71.66s/it]

100
101
102
103
104
105
106
107
108
109
110


100%|██████████| 10/10 [12:00<00:00, 72.04s/it]

111


In [10]:
base_path = '/Users/attar/Desktop/PS/New Folder With Items/Tether-AfterMut-0.20'

output_path = '/Users/attar/Desktop/PS/New Folder With Items/Results'

condition = base_path.split('/')[-1]

ps_data_first_array, ps_data_last_array = process_pulling_data(base_path, condition, output_path, strain)

b_color = '#0B3D91'

color = '#077E97'

PSFS_FTNC_s_het, PSLS_FTNC_s_het,  PSFS_FTNC_melt_s_het, PSLS_FTNC_melt_s_het = process_and_plot_ps_data(ps_data_first_array, ps_data_last_array, output_path, condition, b_color, color)

filter_and_plot_zoomed_ps_data(PSFS_FTNC_melt_s_het, PSLS_FTNC_melt_s_het, base_path, condition, output_path, b_color, color)


100%|██████████| 10/10 [14:20<00:00, 86.09s/it]


In [11]:
base_path = '/Users/attar/Desktop/PS/New Folder With Items/Tether-AfterMut-0.60'

output_path = '/Users/attar/Desktop/PS/New Folder With Items/Results'

condition = base_path.split('/')[-1]

ps_data_first_array, ps_data_last_array = process_pulling_data(base_path, condition, output_path, strain)

b_color = '#0B3D91'

color = '#077E97'

PSFS_FTNC_s_het, PSLS_FTNC_s_het,  PSFS_FTNC_melt_s_het, PSLS_FTNC_melt_s_het = process_and_plot_ps_data(ps_data_first_array, ps_data_last_array, output_path, condition, b_color, color)

filter_and_plot_zoomed_ps_data(PSFS_FTNC_melt_s_het, PSLS_FTNC_melt_s_het, base_path, condition, output_path, b_color, color)


100%|██████████| 10/10 [14:20<00:00, 86.07s/it]
